<a href="https://colab.research.google.com/github/derrodo61/notebooks/blob/main/neo4j_create.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic cypher commands

## Create nodes using MERGE

```
LOAD CSV WITH HEADERS
from 'https://raw.githubusercontent.com/MidnightSkyUniverse/udemyNeo4j/master/data/titles_and_skills.csv'
AS row
MERGE (t:Title {name: row.Title})
```

```
LOAD CSV WITH HEADERS
from 'https://raw.githubusercontent.com/MidnightSkyUniverse/udemyNeo4j/master/data/skills_details.csv'
AS row
MERGE (s:Skill {name: row.Skill})
SET
s.id = row.ID,
s.description = row.Description,
s.category=row.Category
```

```
CREATE (n:Person {name: 'Alice', age: 30})
RETURN n
```

## Create nodes using CREATE

```
CREATE (n:Person {name: 'Alice', age: 30})
RETURN n
```

## Add property

```
LOAD CSV WITH HEADERS
from 'https://raw.githubusercontent.com/MidnightSkyUniverse/udemyNeo4j/master/data/titles_and_skills.csv'
AS row
MATCH (t:Title {name: row.Title})
SET t.skills = split(row.Skills, '|')
```

## Remove property

```
MATCH (t:Title) REMOVE t.skills
```

## Create edges / relations

```
MATCH (t:Title)
UNWIND t.skills AS skill
MERGE (s:Skill {name: skill})
MERGE (t)-[:REQUIRES]->(s)
```

## Add additional labels

```
MATCH (s:Skill{category:'Certification'})
SET s:Skill:Certification
RETURN s.name, labels(s)

MATCH (s:Skill{category:'SpecializedSkill'})
SET s:Skill:SpecializedSkill
RETURN s.name, labels(s)

MATCH (s:Skill) REMOVE s.category
```

## Match

```
MATCH (n) RETURN n

MATCH (n:Skill|Title) RETURN n

MATCH (n:Title) RETURN n LIMIT 25

MATCH (t:Title) RETURN t.name AS name, t.skills AS skills LIMIT 5

MATCH (s:Skill) RETURN DISTINCT s.category

MATCH (n) RETURN LABELs(n) as labels, count(*) as num

MATCH (s:Skill) WHERE s.name = '.NET' RETURN s.name

# case sensitive!
MATCH (s:Skill) WHERE s.name CONTAINS ('Oracle') RETURN s

MATCH (t)-[r:REQUIRES]-(s)
RETURN t,r,s

MATCH (t:Title)-[r:REQUIRES]-(s:Skill)
RETURN t,r,s

PROFILE MATCH (t)-[r:REQUIRES]-(s)
RETURN t,r,s

MATCH (t)-[r:REQUIRES]->(s:Skill)
RETURN DISTINCT LABELS(s), count(*)

PROFILE MATCH (t)-[r:REQUIRES]-(s)
RETURN t,r,s

PROFILE MATCH (t:Title)-[r:REQUIRES]-(s:Skill)
RETURN t,r,s

```

## Schema procedures

```
call db.schema.visualization
call db.schema.nodeTypeProperties
call db.schema.relTypeProperties
```

## Vector index

Create ‘embedding’ property.

APOC has to be installed: https://neo4j.com/docs/apoc/current/installation/

```
LOAD CSV WITH HEADERS
FROM 'https://raw.githubusercontent.com/MidnightSkyUniverse/udemyNeo4j/master/data/embeddings.csv'
AS row
MATCH (s:Skill {name: row.Skill})
CALL db.create.setNodeVectorProperty(s,'embedding',apoc.convert.fromJsonList(row.Embedding))
RETURN count(*)
```

List Skill nodes and property ‘embedding’

```
MATCH (s:Skill) RETURN s.name,
s.embedding
```

Create a vector index with
‘SkillDescription’ as its name, add it to
‘Skill’ node, use ‘embedding’ property
to search through, set 384* as vector
length, use cosine similarity as search
function.

(*) it’s important to know vector length
that you have created. OpenAI vector
is 1536 long. For the purpose of the
project we used Sentence Transformer
from LangChain which generates
vectors 384 long

```
CALL db.index.vector.createNodeIndex(
'skillDescription',
'Skill',
'embedding',
384,
'cosine'
)
```

Show vector index

```
SHOW INDEXES YIELD id, name, type, state,
populationPercent WHERE type = "VECTOR"
```